# VI - Automatic Model Selection from Parametric Sweep using Task Dependencies
In this notebook we will be taking the example from the [Parametric Sweep](04_Parameter_Sweep.ipynb) notebook and automating the entire chain using task dependencies in a single Azure Batch job.

* [Setup](#section1)
* [Configure job](#section2)
* [Submit job](#section3)
* [Download best model](#section4)
* [Delete job](#section5)

<a id='section1'></a>

## Setup

Create a simple alias for Batch Shipyard

In [27]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working

In [28]:
shipyard

Usage: shipyard.py [OPTIONS] COMMAND [ARGS]...

  Batch Shipyard: Provision and Execute Docker Workloads on Azure Batch

Options:
  --version   Show the version and exit.
  -h, --help  Show this message and exit.

Commands:
  cert      Certificate actions
  data      Data actions
  fs        Filesystem in Azure actions
  jobs      Jobs actions
  keyvault  KeyVault actions
  misc      Miscellaneous actions
  pool      Pool actions
  storage   Storage actions


Read in the account information we saved earlier

In [29]:
import json

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)
    
account_info = read_json('account_information.json')

storage_account_key = account_info['storage_account_key']
storage_account_name = account_info['storage_account_name']
IMAGE_NAME = account_info['IMAGE_NAME']
STORAGE_ALIAS = account_info['STORAGE_ALIAS']

<a id='section2'></a>

## Configure Job

As in the previous job we ran on a single node we will be running the job on GPU enabled nodes. The difference here is that depending on the number of combinations we will be creating the same number of tasks. Each task will have a different set of parmeters that we will be passing to our model training script. This parameters effect the training of the model and in the end the performance of the model. The model and results of its evaluation are recorded and stored on the node. At the end of the task the results are pulled into the specified storage container.

In [30]:
from copy import copy
import os
import random
from sklearn.grid_search import ParameterGrid
from toolz import curry, pipe

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

def compose_command(num_convolution_layers, minibatch_size, max_epochs=30):
    cmd_str = ' '.join(("source /cntk/activate-cntk;",
                        "python -u ConvNet_CIFAR10.py",
                        "--datadir $AZ_BATCH_NODE_SHARED_DIR/data",
                        "--num_convolution_layers {num_convolution_layers}",
                        "--minibatch_size {minibatch_size}",
                        "--max_epochs {max_epochs}")).format(num_convolution_layers=num_convolution_layers,
                                                             minibatch_size=minibatch_size,
                                                             max_epochs=max_epochs)
    return 'bash -c "{}"'.format(cmd_str)

@curry
def append_parameter(param_name, param_value, data_dict):
    data_dict[param_name]=param_value
    return data_dict



Create the task template:

In [32]:
_TASK_TEMPLATE = {
    "image": IMAGE_NAME,
    "remove_container_after_exit": True,
    "gpu": True,
    "resource_files": [
            {
                "file_path": "ConvNet_CIFAR10.py",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/ConvNet_CIFAR10.py",
                "file_mode":'0777'
            }
    ],
    "output_data": {
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": "output",
                "source": "$AZ_BATCH_TASK_DIR/wd/Models"
            },
        ]
    },
}

Here we modify the `task_generator` to append an additional parameter to each task definition with a specific `id` numbered from `0`. This is done so we can reference each dependent task in the selection task that must run after each training task has completed successfully.

In [33]:
def task_generator(parameters):
    id = 0
    for params in ParameterGrid(parameters):
        yield pipe(copy(_TASK_TEMPLATE),
                   append_parameter('command', compose_command(**params)),
                   append_parameter('id', str(id)))
        id += 1

Generate the `jobs.json` configuration file

In [34]:
parameters = {
    "num_convolution_layers": [2, 3],
    "minibatch_size": [32, 64]
}

In [35]:
JOB_ID = 'cntk-ps-as-job'

jobs = {
    "job_specifications": [
        {
            "id": JOB_ID,
            "tasks": list(task_generator(parameters))    
        }
    ]
}

num_parameter_sweep_tasks = len(jobs['job_specifications'][0]['tasks'])
print('number of tasks for parametric sweep {}: {}'.format(JOB_ID, num_parameter_sweep_tasks))

number of tasks for parametric sweep cntk-ps-as-job: 4


Now we'll create the Python program to run that performs the best model selection. Note that this code is nearly similar to the code for selecting the best model locally in the [Parameter sweep notebook](04_Parameter_Sweep.ipynb).

In [36]:
%%writefile autoselect.py
import json
import os
import shutil

def read_json(filename):
    with open(filename, 'r') as infile:
        return json.load(infile)

def scandir(basedir):
    for root, dirs, files in os.walk(basedir):
        for f in files:
            yield os.path.join(root, f) 

MODELS_DIR = os.path.join('wd', 'Models')
            
results_dict = {}
for model in scandir(MODELS_DIR):
    if not model.endswith('.json'):
        continue
    key = model.split(os.sep)[2]  # due to MODELS_DIR path change
    results_dict[key] = read_json(model)

# use items() instead of iteritems() as this will be run in python3
tuple_min_error = min(results_dict.items(), key=lambda x: x[1]['test_metric'])
configuration_with_min_error = tuple_min_error[0]
print('task with smallest error: {} ({})'.format(configuration_with_min_error, tuple_min_error[1]['test_metric']))

# copy best model to wd
MODEL_NAME = 'ConvNet_CIFAR10_model.dnn'
shutil.copy(os.path.join(MODELS_DIR, configuration_with_min_error, MODEL_NAME), '.')

Writing autoselect.py


We now need to prepare the file to be uploaded to the Azure Storage account to be referenced in the task:

In [37]:
INPUT_CONTAINER = 'input-autoselect'
OUTPUT_CONTAINER = 'output-autoselect'
UPLOAD_DIR = 'autoselect_upload'

!rm -rf $UPLOAD_DIR
!mkdir -p $UPLOAD_DIR
!mv autoselect.py $UPLOAD_DIR
!ls -alF $UPLOAD_DIR

total 12
drwxr-xr-x  2 nbuser nbuser 4096 Jun 21 15:25 ./
drwx------ 19 nbuser nbuser 4096 Jun 21 15:25 ../
-rw-r--r--  1 nbuser nbuser 1001 Jun 21 15:25 autoselect.py


Alias `blobxfer` and upload it to `INPUT_CONTAINER`:

In [38]:
%alias blobxfer python -m blobxfer

In [39]:
blobxfer $storage_account_name $INPUT_CONTAINER $UPLOAD_DIR --upload --storageaccountkey $storage_account_key

 azure blobxfer parameters [v0.12.1]
             platform: Linux-4.4.0-75-generic-x86_64-with-debian-stretch-sid
   python interpreter: CPython 2.7.11
     package versions: az.common=1.1.5 az.sml=0.20.5 az.stor=0.34.2 crypt=1.9 req=2.18.1
      subscription id: None
      management cert: None
   transfer direction: local->Azure
       local resource: autoselect_upload
      include pattern: None
      remote resource: None
   max num of workers: 24
              timeout: None
      storage account: batcha3dc41fdst
              use SAS: False
  upload as page blob: False
  auto vhd->page blob: False
 upload to file share: False
 container/share name: input-autoselect
  container/share URI: https://batcha3dc41fdst.blob.core.windows.net/input-autoselect
    compute block MD5: False
     compute file MD5: True
    skip on MD5 match: True
   chunk size (bytes): 4194304
     create container: True
  keep mismatched MD5: False
     recursive if dir: True
component strip on up: 1
        r

Now we'll append the `auto-model-selection` task which depends on the prior training tasks. The important properties here are `depends_on_range` which specifies a range of task ids the `auto-model-selection` task depends on. Additionally, this task requires data from the prior run task which is specified in `input_data`.

In [40]:
def generate_input_data_spec(job_id, task_id):
    return {
        "job_id": job_id,
        "task_id": task_id,
        "include": ["wd/Models/*_{}_{}/*".format(task_id, job_id)]
    }

input_data = []
for x in range(0, num_parameter_sweep_tasks):
    input_data.append(generate_input_data_spec(JOB_ID, str(x)))

model_selection_task = {
    "id": "auto-model-selection",
    "command": 'bash -c "source /cntk/activate-cntk; python -u autoselect.py"',
    "depends_on_range": [0, num_parameter_sweep_tasks - 1],
    "image": IMAGE_NAME,
    "remove_container_after_exit": True,
    "input_data": {
        "azure_batch": input_data,
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": INPUT_CONTAINER
            }
        ]
    },
    "output_data": {
        "azure_storage": [
            {
                "storage_account_settings": STORAGE_ALIAS,
                "container": OUTPUT_CONTAINER,
                "include": ["*wd/ConvNet_CIFAR10_model.dnn"],
                "blobxfer_extra_options": "--delete --strip-components 2"
            }
        ]
    }
}

# append auto-model-selection task to jobs
jobs['job_specifications'][0]['tasks'].append(model_selection_task)

In [41]:
write_json_to_file(jobs, os.path.join('config', 'jobs.json'))
print(json.dumps(jobs, indent=4, sort_keys=True))

{
    "job_specifications": [
        {
            "id": "cntk-ps-as-job", 
            "tasks": [
                {
                    "command": "bash -c \"source /cntk/activate-cntk; python -u ConvNet_CIFAR10.py --datadir $AZ_BATCH_NODE_SHARED_DIR/data --num_convolution_layers 2 --minibatch_size 32 --max_epochs 30\"", 
                    "gpu": true, 
                    "id": "0", 
                    "image": "microsoft/cntk:2.0-gpu-python3.5-cuda8.0-cudnn5.1", 
                    "output_data": {
                        "azure_storage": [
                            {
                                "container": "output", 
                                "source": "$AZ_BATCH_TASK_DIR/wd/Models", 
                                "storage_account_settings": "mystorageaccount"
                            }
                        ]
                    }, 
                    "remove_container_after_exit": true, 
                    "resource_files": [
                        {
 

<a id='section3'></a>

## Submit job
Check that everything is ok with our pool before we submit our jobs

In [15]:
shipyard pool list

2017-06-21 10:00:47,791 INFO - pool_id=gpupool [state=PoolState.active allocation_state=AllocationState.steady vm_size=standard_nc6 node_agent=batch.node.ubuntu 16.04 vm_dedicated_count=3 target_vm_dedicated_count=3 vm_low_priority_count=0 target_vm_low_priority_count=0]


Now that we have confirmed everything is working we can execute our job using the command below. 

In [42]:
shipyard jobs add

2017-06-21 15:27:29,203 INFO - Adding job cntk-ps-as-job to pool gpupool
2017-06-21 15:27:29,794 INFO - uploading file /tmp/tmpUzvZEY as u'shipyardtaskrf-cntk-ps-as-job/0.shipyard.envlist'
2017-06-21 15:27:30,060 INFO - uploading file /tmp/tmpMQ8DhB as u'shipyardtaskrf-cntk-ps-as-job/1.shipyard.envlist'
2017-06-21 15:27:30,301 INFO - uploading file /tmp/tmpPE2ZcQ as u'shipyardtaskrf-cntk-ps-as-job/2.shipyard.envlist'
2017-06-21 15:27:30,558 INFO - uploading file /tmp/tmpawAWLm as u'shipyardtaskrf-cntk-ps-as-job/3.shipyard.envlist'
2017-06-21 15:27:31,015 DEBUG - submitting 5 tasks (0 -> 4) to job cntk-ps-as-job
2017-06-21 15:27:31,309 INFO - submitted all 5 tasks to job cntk-ps-as-job


Using the command below we can check the status of our jobs. Once all jobs have an exit code we can continue. You can also view the **heatmap** of this pool on [Azure Portal](https://portal.azure.com) to monitor the progress of this job on the compute nodes under your Batch account.

In [22]:
shipyard jobs listtasks --jobid $JOB_ID

2017-06-21 10:19:56,818 INFO - job_id=cntk-ps-as-job task_id=0 [state=TaskState.completed max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-4283973576_2-20170621t092540z start_time=2017-06-21 10:00:57.299984+00:00 end_time=2017-06-21 10:11:14.302835+00:00 duration=0:10:17.002851 exit_code=0]
2017-06-21 10:19:56,818 INFO - job_id=cntk-ps-as-job task_id=1 [state=TaskState.completed max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-4283973576_3-20170621t092540z start_time=2017-06-21 10:00:56.712625+00:00 end_time=2017-06-21 10:12:26.286121+00:00 duration=0:11:29.573496 exit_code=0]
2017-06-21 10:19:56,819 INFO - job_id=cntk-ps-as-job task_id=2 [state=TaskState.completed max_retries=0 retention_time=10675199 days, 2:48:05.477581 pool_id=gpupool node_id=tvm-4283973576_1-20170621t092540z start_time=2017-06-21 10:10:02.125950+00:00 end_time=2017-06-21 10:18:01.573246+00:00 duration=0:07:59.447296 exit_code=0]
2017-06-21

<a id='section4'></a>

## Download best model
The best performing model from the parametric sweep job should now be saved to our `OUTPUT_CONTAINER` container by the `auto-model-selection` task. Let's save this model in `MODELS_DIR`:

In [23]:
MODELS_DIR = 'auto-selected-model'

Download the best performing model:

In [24]:
blobxfer $storage_account_name $OUTPUT_CONTAINER $MODELS_DIR --remoteresource . --download --storageaccountkey $storage_account_key

 azure blobxfer parameters [v0.12.1]
             platform: Linux-4.4.0-75-generic-x86_64-with-debian-stretch-sid
   python interpreter: CPython 2.7.11
     package versions: az.common=1.1.5 az.sml=0.20.5 az.stor=0.34.2 crypt=1.9 req=2.18.1
      subscription id: None
      management cert: None
   transfer direction: Azure->local
       local resource: auto-selected-model
      include pattern: None
      remote resource: .
   max num of workers: 24
              timeout: None
      storage account: batche2914cdbst
              use SAS: False
  upload as page blob: False
  auto vhd->page blob: False
 upload to file share: False
 container/share name: output-autoselect
  container/share URI: https://batche2914cdbst.blob.core.windows.net/output-autoselect
    compute block MD5: False
     compute file MD5: True
    skip on MD5 match: True
   chunk size (bytes): 4194304
     create container: True
  keep mismatched MD5: False
     recursive if dir: True
component strip on up: 1
        

The best model file (`ConvNet_CIFAR10_model.dnn`) is now ready for use.

In [25]:
!ls -alF $MODELS_DIR

total 4700
drwxr-xr-x  2 nbuser nbuser    4096 Jun 21 10:20 ./
drwx------ 13 nbuser nbuser    4096 Jun 21 10:20 ../
-rw-r--r--  1 nbuser nbuser 4802199 Jun 21 10:20 ConvNet_CIFAR10_model.dnn


<a id='section5'></a>

## Delete job

To delete the job use the command below. Just be aware that this will get rid of all the files created by the job and tasks.

In [43]:
shipyard jobs del -y --termtasks --wait

2017-06-21 15:30:24,436 INFO - Deleting job: cntk-ps-as-job
2017-06-21 15:30:24,436 DEBUG - disabling job cntk-ps-as-job first due to task termination
2017-06-21 15:30:26,047 INFO - Terminating task: 0
cntk-ps-as-job-0
Connection to 52.186.123.38 closed.
2017-06-21 15:30:27,948 INFO - Terminating task: 1
cntk-ps-as-job-1
Connection to 52.186.123.38 closed.
2017-06-21 15:30:29,564 INFO - Terminating task: 2
2017-06-21 15:30:29,949 INFO - Terminating task: 3
cntk-ps-as-job-3
Connection to 52.186.123.38 closed.
2017-06-21 15:30:32,604 INFO - Terminating task: auto-model-selection
2017-06-21 15:30:33,063 DEBUG - waiting for task 0 in job cntk-ps-as-job to terminate
2017-06-21 15:30:33,251 DEBUG - waiting for task 1 in job cntk-ps-as-job to terminate
2017-06-21 15:30:33,434 DEBUG - waiting for task 2 in job cntk-ps-as-job to terminate
2017-06-21 15:30:33,598 DEBUG - waiting for task 3 in job cntk-ps-as-job to terminate
2017-06-21 15:30:33,805 DEBUG - waiting for task auto-model-selection in

## Next Steps
You can proceed to the [Notebook: Clean Up](05_Clean_Up.ipynb) if you are done for now, or proceed to one of the following additional Notebooks:
* [Notebook: Tensorboard Visualization](07_Advanced_Tensorboard.ipynb) - note this requires running this notebook on your own machine
* [Notebook: Parallel and Distributed](08_Advanced_Parallel_and_Distributed.ipynb)
* [Notebook: Keras with TensorFlow](09_Keras_Single_GPU_Training_With_Tensorflow.ipynb)